In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

# Prepare people data frame
df_people = pd.read_csv('Data/demog.csv')
print(df_people.shape)
# Keep only people for which we have age
df_people = df_people[~pd.isna(df_people.age)]
print(df_people.shape)
# Target teenagers
df_people['target_age'] = df_people.age <= 19
df_people = df_people[['userid', 'target_age', 'age']]
df_people.userid = df_people.userid.astype('category')
df_people.head()

(4282857, 15)
(1573876, 15)


,userid,target_age,age
6,631f821671a94f991413e58c2b393b29,False,30.0
7,bdeb2156021c5da9237f5d2667ab590a,False,29.0
12,d9fb05627d4659484703e5827690b9f9,False,27.0
23,adc399722fccda28049cb26c5a795b16,False,29.0
29,18f92f9c1f86b5cd692e8fcc19721b1d,False,27.0


In [2]:
# Prepare likes data frame
df_likes = pd.read_csv('Data/likes.csv', header=None, names=['userid', 'like_id'])
df_likes.userid = df_likes.userid.astype('category')
print(df_likes.shape)

(55775701, 2)


In [3]:
# Prune to keep only pages with more than 1,000 users and users with at least one like
change = True
while change:
    # Keep only likes of people for which we have information
    print("Likes before and after")
    val = df_likes.shape[0]
    print(df_likes.shape)
    df_likes = df_likes[df_likes.userid.isin(df_people.userid)]
    print(df_likes.shape)
    change = val != df_likes.shape[0]

    # Keep only people with at least one like
    print("People before and after")
    val = df_people.shape[0]
    print(df_people.shape)
    df_people = df_people[df_people.userid.isin(df_likes.userid.unique())]
    print(df_people.shape)
    change = (val != df_people.shape[0]) | change
    # Keep only pages with at least 1,000 likes
    print("Pages before and after")
    counts = df_likes.like_id.value_counts()
    val = len(counts)
    high_counts = counts[counts >= 1000]
    change = (val != len(high_counts)) | change
    print(len(counts))
    df_likes = df_likes[df_likes.like_id.isin(high_counts.index.values)]
    print(len(high_counts))

Likes before and after
(55775701, 2)
(37686388, 2)
People before and after
(1573876, 3)
(588610, 3)
Pages before and after
15914
10822
Likes before and after
(33873012, 2)
(33873012, 2)
People before and after
(588610, 3)
(587745, 3)
Pages before and after
10822
10822
Likes before and after
(33873012, 2)
(33873012, 2)
People before and after
(587745, 3)
(587745, 3)
Pages before and after
10822
10822


In [4]:
# Get labels
labels = df_people.set_index('userid').loc[sorted(df_likes.userid.unique())].target_age.values
ages = df_people.set_index('userid').loc[sorted(df_likes.userid.unique())].age.values
# Build sparse matrix
users_c = np.array(sorted(df_likes.userid.unique()))
pages_c = np.array(sorted(df_likes.like_id.unique()))
row = pd.Categorical(df_likes.userid, categories=users_c, ordered=True).codes
col = pd.Categorical(df_likes.like_id, categories=pages_c, ordered=True).codes
sparse_mat = csr_matrix((np.ones(df_likes.shape[0]), (row, col)), shape=(users_c.size, pages_c.size))
sparse_mat

<587745x10822 sparse matrix of type '<class 'numpy.float64'>'
	with 33873012 stored elements in Compressed Sparse Row format>

In [5]:
# Build machine learning model
X_train, X_test, y_train, y_test, age_train, age_test = train_test_split(sparse_mat, labels, ages,
                                                                         test_size=0.3, random_state=42)

model = LogisticRegression(random_state=42, solver='lbfgs').fit(X_train, y_train)
fpr, tpr, thresholds = metrics.roc_curve(y_test, model.predict_proba(X_test)[:, 1])
metrics.auc(fpr, tpr)

C:\Users\ferlo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


0.8727784573849582

In [6]:
# Load names of facebook pages
df_names = pd.read_csv('Data/fb_like.csv', encoding='latin-1', error_bad_lines=False, warn_bad_lines=False)
print(df_names.shape)
df_names.head()
df_names = df_names[df_names.like_id.isin(pages_c)]
print(df_names.shape)
df_names = df_names.set_index('like_id')
df_names.head()

(19041200, 2)
(10821, 2)


,name
like_id,
9254817079,photography
103144503058854,Poetry
104046862964234,Writing
105667052799444,Piano
106059522759137,English


In [7]:
# Get threshold
probs = model.predict_proba(X_test)[:, 1]
threshold = np.percentile(probs, 95)
print(threshold)
# Keep only people with probabilities above threshold
top_x = X_test[probs > threshold, :]
top_y = y_test[probs > threshold]
top_age = age_test[probs > threshold]
# Take a look at average number of likes
print(top_x.sum(axis=1).mean())
print(np.median(np.array(top_x.sum(axis=1))))
# Keep interesting errors
top_x = top_x[top_age >= 30]
top_y = top_y[top_age >= 30]
top_age = top_age[top_age >= 30]
top_x = top_x[top_age <= 50]
top_y = top_y[top_age <= 50]
top_age = top_age[top_age <= 50]
print("Errors of people between 30 and 50 years")
print(top_x.shape[0])
# Take a look at probabilities
print(model.predict_proba(top_x)[:, 1])
print(top_x.sum(axis=1).reshape(1, -1))

0.794988674752503
276.2428263581717
157.0
Errors of people between 30 and 50 years
99
[0.98169237 0.99896217 0.97713391 0.99986298 0.9476107  0.85177353
 0.9951256  0.9983207  0.95869546 0.98730223 0.94229269 0.99100626
 0.99347797 0.99317242 0.8136403  0.96125115 0.92354525 0.96041856
 0.99998922 0.81137904 0.85491682 0.99958716 0.99105258 0.92894244
 0.81376887 0.99999999 0.80162073 0.94488282 0.95440956 0.9763705
 0.86121731 0.94134376 0.96671992 0.89188035 0.8974984  0.99354177
 0.8932332  0.99511592 0.85857827 0.8316827  0.99988883 0.80858969
 0.8293599  0.99472555 0.94824506 0.97017833 0.99999921 0.99688985
 0.83159057 0.97975629 0.88912899 0.89665586 0.96513789 0.99885476
 0.99813589 0.8396682  0.98670352 0.99598254 0.999205   0.87260677
 0.99463161 0.98667583 0.85896881 0.84546594 0.94797361 0.99897486
 0.80712154 0.96552465 0.87577587 0.99979524 0.8072721  0.9105793
 0.85879328 0.98755166 0.99885857 0.99957779 0.90147285 0.99894757
 0.95510547 0.85074211 0.85197809 0.99622463 

In [8]:
# 37% of targets have made at least 227 likes
targets = model.predict_proba(X_test)[:, 1] > threshold
np.percentile(X_test[targets, :].sum(axis=1), 63)

227.0

In [9]:
model.predict_proba(top_x)[42, 1]

0.829359899218118

In [10]:
import shap

# Candidates 12, 3, 15, 21
# Candidates 32, 74, 88, 39, 41, 46

def make_decision(data):
    return (model.predict_proba(data)[:, 1] > threshold).astype(float)

#for i in range(99):
i = 32
print("----------------------------------------------")
print("Position: ", i)
top_x_guy = top_x[np.full(5, i), :]
np.random.seed(0)
default = csr_matrix(np.zeros(X_test.shape[1]))
shap_explainer = shap.KernelExplainer(make_decision, default)
shap_values = shap_explainer.shap_values(top_x_guy, nsamples=4000, l1_reg='aic')

# Top 5 predictive features are different every time. 
for i in range(shap_values.shape[0]):
    i_vals = (-shap_values[i, :]).argsort()
    print(df_names.loc[pages_c[i_vals[:5]]].name)
    print(shap_values[i, i_vals[:5]])

----------------------------------------------
Position:  32


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:10<00:00,  2.02s/it]


like_id
23625182151        Call of Duty: World at War
76377189413                        Heidi Klum
54244944269                        Demi Moore
42798291365             SpongeBob SquarePants
282322361806841                      The Sims
Name: name, dtype: object
[0.14280135 0.11432135 0.10700707 0.0978031  0.07529212]
like_id
23625182151        Call of Duty: World at War
78041983445                        Chocolate!
7427438063                         The Sims 2
282322361806841                      The Sims
54244944269                        Demi Moore
Name: name, dtype: object
[0.10451007 0.10240373 0.09898086 0.09107538 0.08861605]
like_id
54244944269                        Demi Moore
23625182151        Call of Duty: World at War
76377189413                        Heidi Klum
282322361806841                      The Sims
56470448215                              Puma
Name: name, dtype: object
[0.12183726 0.11313927 0.10255483 0.10190203 0.07478998]
like_id
76377189413            Heidi 

In [11]:
# Number of websites liked by the guy
top_x_guy.sum(axis=1)

matrix([[221.],
        [221.],
        [221.],
        [221.],
        [221.]])

In [12]:
# See how long it takes for our method to find one explanation
import explainer
import time

def scoring_function(data):
    return model.predict_proba(data)[:, 1]

# Get evidence-based explanations
explain = explainer.Explainer(scoring_function, default, prune=False)
t0 = time.time()
explanations = explain.explain(top_x_guy[0, :], thresholds=float(threshold), max_ite=50)
t1 = time.time()
print(t1-t0)
print("Explanations found")
print(len(explanations[0]))
print("Length of explanation")
print(len(explanations[0][0]))
print("First explanation")
print(df_names.loc[pages_c[explanations[0][0]]])
print(df_names.loc[pages_c[explanations[0][1]]])
print(df_names.loc[pages_c[explanations[0][2]]])
print(df_names.loc[pages_c[explanations[0][3]]])
print(df_names.loc[pages_c[explanations[0][4]]])

0.11967897415161133
Explanations found
45
Length of explanation
4
First explanation
                                   name
like_id                                
23625182151  Call of Duty: World at War
54244944269                  Demi Moore
76377189413                  Heidi Klum
78041983445                  Chocolate!
                                       name
like_id                                    
23625182151      Call of Duty: World at War
54244944269                      Demi Moore
76377189413                      Heidi Klum
282322361806841                    The Sims
                                   name
like_id                                
23625182151  Call of Duty: World at War
39399781765              Boston Red Sox
54244944269                  Demi Moore
76377189413                  Heidi Klum
                                       name
like_id                                    
23625182151      Call of Duty: World at War
54244944269                      Demi Mo

In [13]:
from functools import reduce

sample_size = 500
np.random.seed(0)
# Get sample of targeted observations to explain
X_targeted = X_test[probs > threshold, :]
X_sample = X_targeted[np.random.choice(X_targeted.shape[0], sample_size, replace=False), :]
X_to_explain = X_sample[np.arange(X_sample.shape[0]).repeat(5), :]
# Obtain their SHAP values
#shap_values = shap_explainer.shap_values(X_to_explain, nsamples=4000, l1_reg='aic')
#np.save("back_up_shap_values.npy", shap_values)
shap_values = np.load("Data/back_up_shap_values.npy")
# Find top 5 most informative features
most_informative_f = np.argsort(shap_values, axis=1)[:, -5:]
# Compute number of matches
sample_matches = list()
for i in range(sample_size):
    matches = reduce(np.intersect1d, most_informative_f[(i*5):((i+1)*5), :]).size
    sample_matches.append(matches)
    
print(sample_matches)

[0, 2, 3, 1, 1, 3, 4, 0, 3, 2, 1, 3, 2, 1, 1, 0, 1, 5, 0, 3, 3, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 2, 2, 2, 2, 2, 3, 1, 0, 0, 4, 1, 0, 2, 1, 3, 3, 3, 0, 5, 1, 3, 0, 1, 3, 4, 1, 3, 4, 3, 3, 3, 5, 1, 3, 3, 2, 1, 2, 2, 2, 1, 0, 2, 3, 5, 3, 3, 1, 0, 0, 1, 1, 1, 1, 4, 2, 0, 3, 4, 3, 3, 4, 3, 1, 0, 5, 2, 2, 4, 1, 2, 3, 3, 2, 4, 2, 0, 1, 0, 2, 3, 1, 3, 3, 1, 2, 4, 0, 0, 3, 1, 1, 4, 4, 0, 3, 2, 2, 2, 2, 0, 0, 0, 3, 3, 1, 1, 2, 4, 3, 0, 5, 5, 2, 3, 3, 3, 4, 3, 4, 1, 2, 2, 0, 1, 0, 3, 2, 1, 2, 0, 5, 3, 1, 2, 2, 0, 2, 0, 1, 4, 0, 2, 1, 4, 3, 3, 1, 0, 0, 0, 0, 1, 3, 0, 0, 0, 3, 2, 0, 0, 0, 4, 2, 3, 3, 1, 1, 0, 0, 2, 1, 0, 3, 1, 0, 4, 3, 1, 2, 3, 3, 0, 4, 1, 4, 0, 0, 0, 1, 3, 1, 0, 2, 2, 0, 4, 1, 4, 1, 0, 0, 1, 1, 0, 2, 0, 3, 4, 0, 0, 2, 1, 2, 4, 1, 3, 4, 0, 3, 3, 0, 3, 3, 3, 2, 4, 3, 3, 0, 2, 4, 2, 0, 1, 3, 0, 2, 1, 1, 2, 2, 1, 4, 2, 2, 0, 3, 4, 1, 0, 1, 3, 4, 0, 1, 4, 1, 3, 1, 2, 0, 2, 4, 1, 1, 4, 0, 3, 4, 0, 2, 3, 1, 2, 2, 0, 5, 2, 3, 4, 1, 2, 0, 2, 0, 1, 1, 1, 0, 3, 0, 2, 2, 0, 1, 4, 1, 2, 0, 0, 4, 

In [14]:
likes_count = np.array(X_sample.sum(axis=1)).flatten()
cut_points = list()
for i in range(1, 5):
    cut_points.append(np.percentile(likes_count, i*20))
cut_points.append(np.inf)

min_likes = 0
sample_matches = np.array(sample_matches) 
for cp in cut_points:
    print("Upper limit: ", cp)
    selected_obs = (likes_count >= min_likes) & (likes_count < cp)
    print(sample_matches[selected_obs].mean())
    min_likes = cp

Upper limit:  60.80000000000001
2.87
Upper limit:  114.60000000000002
2.37
Upper limit:  202.0
2.0202020202020203
Upper limit:  442.2000000000003
1.386138613861386
Upper limit:  inf
0.47


In [ ]:
# Get counterfactual explanations
explainer.Explainer(scoring_function, default, prune=False)

t0 = time.time()
sample_e = explain.explain(X_sample, thresholds=float(threshold), max_ite=50, stop_at_first=True)
t1 = time.time()
print(t1-t0)

sample_lens = np.array([len(e[0]) for e in sample_e])
min_likes = 0
for cp in cut_points:
    print("Upper limit: ", cp)
    selected_obs = (likes_count >= min_likes) & (likes_count < cp)
    print(sample_lens[selected_obs].mean())
    min_likes = cp

In [ ]:
# Check out real quantiles
likes_count = X_targeted.sum(axis=1)

print(np.percentile(likes_count, 20))
print(np.percentile(likes_count, 40))
print(np.percentile(likes_count, 60))
print(np.percentile(likes_count, 80))

In [ ]:
from scipy.stats import norm

# Define Cost Function
likes_per_page = np.array(sparse_mat.sum(axis=0)).reshape(-1)
costs = norm.cdf((np.mean(likes_per_page) - likes_per_page) / np.std(likes_per_page))
costs += np.full(X_train.shape[1], 0.1)

def likes_cost_func(original_obs, new_obs):
    return (original_obs - new_obs).dot(costs)

# Method to print explanation details
def print_explanation(e):
    for p in e:
        print(df_names.loc[pages_c[p]].values[0], "| Likes :", int(likes_per_page[p]))

#  Find explanations with/without costs
# Good candidates: 243, 457, 11, 148, 194
ixs = [194, 148, 457]
t = float(threshold)
for i in ixs:
    # With cost
    cost_e = explain.explain(X_sample[i, :], thresholds=t, stop_at_first=True, cost_func=likes_cost_func)[0][0]
    # No cost
    no_cost_e = explain.explain(X_sample[i, :], thresholds=t, stop_at_first=True)[0][0]
    print("ID: ", i)
    print("Explanation WITHOUT costs")
    print_explanation(no_cost_e)
    print("---")
    print("Explanation WITH costs")
    print_explanation(cost_e)
    print("*************************************************")

Empirical Case 1: Explaining decisions rather than predictions


Empirical Case 2: High-dimensional and context specific explanations

Probabilistic approaches are unreliable when we are dealing with high dimensional spaces. Someone old got targeted with teen stuff, wants to know why. One could show the person the top most informative sites that led to the decision. Turns out those sites are very sensible to the probabilistic procedure. Not a single site appears every time. 

Systematic analysis. Make four groups of people (only people with at least 15 likes), each one for a quantile. For each group compute SHAP values 5 times. For each observation, count the number of pages that appear in all 5 SHAP values. Get percentage for each (0, 1, 2, 3, 4, and 5 matches). Also record the time it took to compute explanations. 

As the number of likes increases, it becomes increasingly unlikely to get consistent results, even if the goal is to find only the top most informative attributes rather than estimating precise weights. 

Our case, an explanation can be found very fast. Do the same analysis showing the average length of explanation for each group.

Problem is that, there are so many features that some of them may not 

(EXAMPLE 1: Facebook Likes? Predict if someone is young.)
- Analysis for fast (vs precise) explanations (case 2). Look for young (or old) person with many likes. Compute SHAP several times to get a distribution of the values. Get average computation time. Compare to computation of EDC (faster, consistent)
- Analysis for "cheapest" explanations (case 3). Show explanations (many likes) when not sorting by popularity. Compare when sorting by popularity.

(EXAMPLE 2: Churn data)
- Incorporate decisions from multiple models. Predict monthly charge and whether the person will churn. Idea: Look for guy where top regression feature and top probability feature are different from top combined feature. Again, show how several features influence expected loss, but only a few are necessary to change decision.